<a href="https://colab.research.google.com/github/howard-haowen/NLP-demos/blob/main/gensim_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path

In [2]:
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
cm.update(
    "rise",
    {
        "autolaunch": False,
        "enable_chalkboard": True,
        "scroll": True,
        "slideNumber": True,
        "controls": True,
        "progress": True,
        "history": True,
        "center": True,
        "width": "100%",
        "height": "100%",
        "theme": "beige",
        "transition": "concave",
        "start_slideshow_at": "selected"
     }
)

{'autolaunch': False,
 'center': True,
 'controls': True,
 'enable_chalkboard': True,
 'height': '100%',
 'history': True,
 'progress': True,
 'scroll': True,
 'slideNumber': True,
 'start_slideshow_at': 'selected',
 'theme': 'beige',
 'transition': 'concave',
 'width': '100%'}

## Dependencies

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install pyldavis
!pip install nltk

In [4]:
# NLP
import spacy
# corpus
import nltk
# topic modelling
import gensim
from gensim.models import LdaModel, HdpModel
from gensim.corpora import Dictionary
# visualization
import pyLDAvis.gensim_models
# tabularization of data
import pandas as pd
# others
import warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


## Download the corpus

In [5]:
nltk.download('inaugural')
from nltk.corpus import inaugural

[nltk_data] Downloading package inaugural to /root/nltk_data...
[nltk_data]   Unzipping corpora/inaugural.zip.


In [6]:
file_ids = inaugural.fileids()
len(file_ids)

59

In [7]:
inaugural.raw(file_ids[0])

'Fellow-Citizens of the Senate and of the House of Representatives:\n\nAmong the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the 14th day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years -- a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications, could not

In [16]:
def inaugura2df(file_ids):
    years = [id[:4] for id in file_ids]
    presidents = [id[5:].replace(".txt", "") for id in file_ids]
    texts = [inaugural.raw(id) for id in file_ids]
    df = pd.DataFrame(
        {
            "year": years,
            "president": presidents,
            "text": texts,
        }
    )
    df.set_index("year", inplace=True)
    df.index = df.index.astype(int)
    return df

In [17]:
df = inaugura2df(file_ids)
df

,president,text
year,,
1789,Washington,Fellow-Citizens of the Senate and of the House...
1793,Washington,"Fellow citizens, I am again called upon by the..."
1797,Adams,"When it was first perceived, in early times, t..."
1801,Jefferson,Friends and Fellow Citizens:\n\nCalled upon to...
1805,Jefferson,"Proceeding, fellow citizens, to that qualifica..."
1809,Madison,Unwilling to depart from examples of the most ...
1813,Madison,About to add the solemnity of an oath to the o...
1817,Monroe,I should be destitute of feeling if I was not ...
1821,Monroe,"Fellow citizens, I shall not attempt to descri..."


## Preprocess data

In [10]:
nlp = spacy.load("en_core_web_sm")

## Build a topic model

In [13]:
# corpus = 
# dictionary = 
# model = 

## Visualize the built model

In [14]:
pyLDAvis.enable_notebook()

In [15]:
# pyLDAvis.gensim_models.prepare(model, corpus, dictionary)